# Démonstration de l'échantillonnage et non respect du théorème de Shannon.

In [ ]:
# On peut générer une page web qui ne contient que l'affichage avec voila, qui est installé
# dans l'environnement : 
# voila notebook.ipynb

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, IntText, VBox, HBox, Layout
from IPython.display import display

# --- Paramètres de l'Affichage Spectral Fixe ---
F_LIM_MIN = -50.0
F_LIM_MAX = 50.0

# --- Fonction principale d'Analyse et de Plot ---
def visualize_aliasing(F_signal, F_s, T_total, N_fft_points):
    """
    Génère le signal sinusoïdal, l'échantillonne et affiche les domaines
    temporel et spectral, avec une échelle de fréquence fixe [-50 Hz, 50 Hz].
    """
    
    # Validation minimale
    if F_s <= 0 or F_signal <= 0 or N_fft_points <= 0 or T_total <= 0:
        print("Les paramètres doivent être positifs.")
        return

    # --- Paramètres calculés ---
    N_sampled = N_fft_points
    T_sample = N_sampled / F_s  # Durée totale d'échantillonnage
    dt_sample = 1 / F_s         # Pas de temps d'échantillonnage
    F_nyquist = F_s / 2         # Fréquence de Nyquist

    # --- Préparation des signaux ---
    
    # Signal continu (pour la référence)
    F_s_high = 100 * F_s if F_s > 0 else 1000
    N_high = int(F_s_high * T_sample)
    t_high = np.linspace(0, T_sample, N_high, endpoint=False)
    signal_analog = np.sin(2 * np.pi * F_signal * t_high)
    
    # Signal échantillonné
    t_sampled = np.linspace(0, T_sample, N_sampled, endpoint=False)
    signal_sampled = np.sin(2 * np.pi * F_signal * t_sampled)

    # --- 1. Domaine Temporel ---
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    ax1.plot(t_high, signal_analog, label='Signal Analogique ($F_0$)', color='gray', linestyle='-')
    ax1.plot(t_sampled, signal_sampled, 'ro', label=f'Points Échantillonnés ($F_e$)', markersize=4)
    ax1.plot(t_sampled, signal_sampled, 'r--', alpha=0.5) 
    
    ax1.set_title(f"Domaine Temporel | $F_0$={F_signal:.1f} Hz, $F_e$={F_s:.1f} Hz")
    ax1.set_xlabel('Temps (s)')
    ax1.set_ylabel('Amplitude')
    ax1.set_xlim(0, T_total) # Zoom
    ax1.grid(True)
    ax1.legend(loc='upper right')
    
    # Affichage de l'aliasing
    if F_signal > F_nyquist:
        F_alias = np.abs(F_signal - np.round(F_signal / F_s) * F_s)
        if F_alias < 0.001 and F_signal > F_nyquist:
             F_alias = F_s - F_signal
             
        ax1.text(0.5, 0.95, 
                 f"⚠️ ALIASING: $F_0 > F_e/2$. Fréquence apparente: {F_alias:.2f} Hz", 
                 transform=ax1.transAxes, 
                 color='red', 
                 fontsize=12, 
                 bbox=dict(facecolor='yellow', alpha=0.5))
    
    
    # --- 2. Domaine Spectral (Périodisation) ---
    
    # Calcul de la TFD
    spectrum = np.fft.fft(signal_sampled)
    magnitude_spectrum = np.abs(spectrum) / N_sampled
    
    # Création du vecteur de fréquences et centrage (Principal période: [-Fe/2, Fe/2])
    frequencies = np.fft.fftfreq(N_sampled, d=dt_sample)
    frequencies_centered = np.fft.fftshift(frequencies)
    magnitude_centered = np.fft.fftshift(magnitude_spectrum)
    
    # Résolution fréquentielle (pas entre deux points du spectre)
    dF = F_s / N_sampled 
    
    # --- Tracé du spectre périodisé ---
    
    ax2.set_title(rf'Domaine Spectral (Spectre Périodisé)')
    
    # Tracé des copies périodiques pour illustrer la périodisation, même si elles sortent du zoom [-50, 50]
    
    # On calcule une plage étendue de -2 * Fe à 2 * Fe pour s'assurer que les pics bleus sont visibles
    MAX_SHIFT = int(np.ceil(F_LIM_MAX / F_s)) + 1 if F_s > 0 else 2 

    # Tracé des copies périodiques
    for k in range(-MAX_SHIFT, MAX_SHIFT + 1):
        is_principal = (k == 0)
        
        ax2.plot(
            frequencies_centered + k * F_s, 
            magnitude_centered, 
            color='green' if is_principal else 'green', 
            linestyle='-' if is_principal else '--',
            alpha=1 if is_principal else 0.5,
            label=r'Spectre Principal' if is_principal else None
        )
    
    # --- Lignes de Référence ---
    
    # Fréquence de Nyquist (limite de la bande principale)
    ax2.axvline(F_nyquist, color='red', linestyle='--', label=rf'Fréquence de Nyquist $\pm F_e/2$ ({F_nyquist:.1f} Hz)')
    ax2.axvline(-F_nyquist, color='red', linestyle='--')
    
    # Fréquence d'échantillonnage (centre des copies)
    ax2.axvline(F_s, color='gray', linestyle=':', label=rf'Fréquence d\'échantillonnage $\pm F_e$')
    ax2.axvline(-F_s, color='gray', linestyle=':')
    
    # Position du pic idéal (dans le spectre analogique/continu)
    
    # Nous traçons F0 et ses images qui tombent dans la plage [-50, 50]
    for k in range(-MAX_SHIFT, MAX_SHIFT + 1):
        F_image_pos = F_signal + k * F_s
        F_image_neg = -F_signal + k * F_s
        
        # Afficher la ligne si elle est dans la plage visible
        #if F_image_pos >= F_LIM_MIN and F_image_pos <= F_LIM_MAX:
        #     ax2.axvline(F_image_pos, color='blue', linestyle='-.', alpha=0.7, label=rf'Images $\pm F_0$ (par $\pm k F_e$)' if k == 0 else None)
        #if F_image_neg >= F_LIM_MIN and F_image_neg <= F_LIM_MAX:
        #     ax2.axvline(F_image_neg, color='blue', linestyle='-.', alpha=0.7)


    # --- Mise en évidence de l'Aliasing ---
    if F_signal > F_nyquist:
        F_alias_val = np.abs(F_signal - np.round(F_signal / F_s) * F_s)
        if F_alias_val < 0.001 and F_signal > F_nyquist:
             F_alias_val = F_s - F_signal

        # La fréquence repliée est le pic qui est effectivement observé dans la bande principale [-Fe/2, Fe/2]
        #ax2.axvline(F_alias_val, color='red', linestyle='-', linewidth=2, label=rf'Fréquence repliée $F_{{alias}}$ ({F_alias_val:.2f} Hz)')
        #ax2.axvline(-F_alias_val, color='red', linestyle='-', linewidth=2)
    
    ax2.set_xlabel('Fréquence (Hz)')
    ax2.set_ylabel('Magnitude')
    
    # FIXER LA LIMITE À [-50 Hz, 50 Hz]
    ax2.set_xlim(F_LIM_MIN, F_LIM_MAX) 
    ax2.set_ylim(0, 0.5)
    ax2.grid(True)
    ax2.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

# --- Configuration des Widgets (Plages ajustées pour la fenêtre [-50, 50]) ---

# 1. Fréquence du Signal F0
slider_f_signal = FloatSlider(value=5.0, min=1.0, max=10.0, step=0.5, description=r'$F_0$ (Hz):')

# 2. Fréquence d'Échantillonnage Fe
# On va jusqu'à 60 Hz pour couvrir des cas où Fe/2 > 50 Hz, ou Fe est petit (ex: 10 Hz) pour voir l'aliasing dans le zoom.
slider_f_s = FloatSlider(value=25.0, min=5.0, max=25.0, step=0.5, description=r'$F_e$ (Hz):')

# 3. Durée d'Affichage Temporelle T_total (Zoom sur le temps)
slider_t_total = FloatSlider(value=2, min=0.1, max=5.0, step=0.1, description=r'$T_{affichage}$ (s):')

# 4. Nombre de points FFT (Définit N et donc la durée totale échantillonnée T_sample)
# text_n_fft = IntText(value=512, min=64, max=4096, step=64, description='N points FFT:', style={'description_width': 'initial'})


# Organisation des widgets
controls_time = HBox([slider_f_signal, slider_f_s])
# controls_plot = HBox([slider_t_total, text_n_fft])
controls_plot = HBox([slider_t_total])
widget_layout = VBox([controls_time, controls_plot])

# Création de l'interface interactive
interactive_plot = interactive(
    visualize_aliasing, 
    F_signal=slider_f_signal, 
    F_s=slider_f_s,
    T_total=slider_t_total,
    N_fft_points=8192
)

# Affichage
print("⚙️ Contrôles Interactifs du Signal et de l'Analyse Spectrale")
print(f"L'échelle spectrale est fixe entre {F_LIM_MIN} Hz et {F_LIM_MAX} Hz.")
display(widget_layout)
interactive_plot.update()
display(interactive_plot.children[-1])

⚙️ Contrôles Interactifs du Signal et de l'Analyse Spectrale
L'échelle spectrale est fixe entre -50.0 Hz et 50.0 Hz.


Output()